In [1]:
import pandas as pd 
import requests 
import numpy as np 
TorontoRD=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0] 
TorontoRD['Borough'] = TorontoRD['Borough'].replace('Not assigned', np.nan)
TorontoRD = TorontoRD.dropna(axis=0, subset=['Borough']) 
TorontoRD['Neighbourhood'] = TorontoRD['Neighbourhood'].replace('Not assigned', np.nan)
TorontoRD.Neighbourhood.fillna(TorontoRD.Borough,inplace=True) 
TorontoRD = TorontoRD.groupby(by=['Postcode','Borough'],sort = False).agg(lambda x: ','.join(x))
TorontoRD=TorontoRD.reset_index() 
Toronto_df=TorontoRD 
path='http://cocl.us/Geospatial_data'
Toronto_new=pd.read_csv(path)
Toronto_df=Toronto_df.sort_values(by=['Postcode'], axis=0)
Toronto_df.reset_index(inplace=True)
Toronto_df.drop('index', inplace=True, axis=1)
Toronto_df[['Latitude','Longitude']]=Toronto_new[['Latitude','Longitude']]
Toronto_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [2]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

In [17]:
from pandas.io.json import json_normalize 

Let's explore Downtown Toronto borough in the above dataframe and introduce new dataform.

In [7]:
Downtown_Toronto = Toronto_df[Toronto_df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
3,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
5,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
9,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752


Now Let's explore the first neighbourhood in DownTown Toront, i.e. Rosedale.

In [11]:
neighborhood_latitude = Downtown_Toronto.iloc[0]['Latitude'] # neighborhood latitude value
neighborhood_longitude = Downtown_Toronto.iloc[0]['Longitude'] # neighborhood longitude value

neighborhood_name = Downtown_Toronto.iloc[0]['Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rosedale are 43.6795626, -79.37752940000001.


Now, let's get the top 50 venues that are in Rosedale within a radius of 1 km. We need to get this information from FourSquare so we need to define our Client_ID, Client_secret and version first.

In [20]:
CLIENT_ID = '40QYIXJHZ1SS0ISI3HBHDECMGEHIGN0VMKQUH4YKFN0UCLYE' 
CLIENT_SECRET ='C30ZV1NMM4YFE5Q4WUMPYRPW0DB23EHI1HY2LDFBUJ5ZQKWV' 
VERSION = '20180605' 
LIMIT = 50 
radius = 1000 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

We need to examine the result by sending Get request.In order to examine furthure we need to extract the category of each venue.

In [21]:
results = requests.get(url).json()
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now let's see the what kinda of venues are aroun Rosedale neighbourhood in Downtown district of Toronto. There are grocery stores, restaurants, parks, etc.

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Summerhill Market,Grocery Store,43.686265,-79.375458
1,Toronto Lawn Tennis Club,Athletics & Sports,43.680667,-79.388559
2,Black Camel,BBQ Joint,43.677016,-79.389367
3,Tinuno,Filipino Restaurant,43.671281,-79.374920
4,Craigleigh Gardens,Park,43.678099,-79.371586
5,Maison Selby,Bistro,43.671232,-79.376618
6,Pie Squared,Pie Shop,43.672143,-79.377856
7,Starbucks,Coffee Shop,43.671082,-79.380756
8,Manulife Financial,Office,43.672070,-79.382449
9,Booster Juice,Smoothie Shop,43.671566,-79.378581


We can check how many venues were found by Foursquare in 1km radius of Rosedale neighbourhood.

In [26]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

28 venues were returned by Foursquare.


We can repeat the same process for other neighbourhoods in Downtown Toronto by defining the following getNearbyVenues function.

In [29]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto['Neighbourhood'],
                                   latitudes=Downtown_Toronto['Latitude'],
                                   longitudes=Downtown_Toronto['Longitude']
                                  )

Checking the size of venues in Downtown Toronto.

In [28]:
print(Downtown_Toronto_venues.shape)
Downtown_Toronto_venues.head(10)

(831, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,43.679563,-79.377529,Toronto Lawn Tennis Club,43.680667,-79.388559,Athletics & Sports
2,Rosedale,43.679563,-79.377529,Black Camel,43.677016,-79.389367,BBQ Joint
3,Rosedale,43.679563,-79.377529,Tinuno,43.671281,-79.374920,Filipino Restaurant
4,Rosedale,43.679563,-79.377529,Craigleigh Gardens,43.678099,-79.371586,Park
5,Rosedale,43.679563,-79.377529,Maison Selby,43.671232,-79.376618,Bistro
6,Rosedale,43.679563,-79.377529,Pie Squared,43.672143,-79.377856,Pie Shop
7,Rosedale,43.679563,-79.377529,Starbucks,43.671082,-79.380756,Coffee Shop
8,Rosedale,43.679563,-79.377529,Manulife Financial,43.672070,-79.382449,Office
9,Rosedale,43.679563,-79.377529,Booster Juice,43.671566,-79.378581,Smoothie Shop


Let's check how many venues were returned for each Borough

In [63]:
Downtown_Toronto_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
"Adelaide,King,Richmond",50,50,50,50,50,50
Berczy Park,50,50,50,50,50,50
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",38,38,38,38,38,38
Central Bay Street,50,50,50,50,50,50
"Chinatown,Grange Park,Kensington Market",50,50,50,50,50,50
Christie,50,50,50,50,50,50
Church and Wellesley,50,50,50,50,50,50
"Commerce Court,Victoria Hotel",50,50,50,50,50,50


So in total 831 venes were found.Let's find out how many unique categories can be curated from all the returned venues.

In [46]:
print('There are {} uniques categories.'.format(len(Downtown_Toronto_venues['Venue Category'].unique())))

There are 180 uniques categories.


In [ ]:
Now analyzing Each Borough.And find out the size of our data.

In [59]:
Downtown_Toronto_onehot = pd.get_dummies(Downtown_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
Downtown_Toronto_onehot['Borough'] = Downtown_Toronto_venues['Borough'] 
fixed_columns = [Downtown_Toronto_onehot.columns[-1]] + list(Downtown_Toronto_onehot.columns[:-1])
Downtown_Toronto_onehot = Downtown_Toronto_onehot[fixed_columns]
Downtown_Toronto_onehot.shape

(831, 181)

Next, let's group rows by borough and by taking the mean of the frequency of occurrence of each catego

In [60]:
Downtown_Toronto_grouped = Downtown_Toronto_onehot.groupby('Borough').mean().reset_index()
Downtown_Toronto_grouped

,Borough,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Thai Restaurant,Theater,Theme Restaurant,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.060000,0.00,0.00,0.00,0.00,0.00,0.04,...,0.020000,0.020000,0.00,0.000000,0.000000,0.00,0.02,0.00,0.00,0.00
1,Berczy Park,0.000000,0.000000,0.000000,0.00,0.00,0.02,0.00,0.00,0.00,...,0.020000,0.000000,0.00,0.000000,0.000000,0.00,0.02,0.00,0.00,0.00
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.066667,0.066667,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.00,0.066667,0.000000,0.00,0.00,0.00,0.00,0.00
3,"Cabbagetown,St. James Town",0.000000,0.000000,0.026316,0.00,0.00,0.00,0.00,0.00,0.00,...,0.026316,0.026316,0.00,0.000000,0.026316,0.00,0.00,0.00,0.00,0.00
4,Central Bay Street,0.000000,0.000000,0.000000,0.00,0.00,0.02,0.02,0.00,0.00,...,0.020000,0.000000,0.00,0.000000,0.000000,0.00,0.02,0.00,0.00,0.02
5,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.02,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.06,0.04,0.02,0.00
6,Christie,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.02,0.00,0.02,0.00
7,Church and Wellesley,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.02,0.00,...,0.040000,0.000000,0.02,0.000000,0.000000,0.00,0.00,0.02,0.00,0.02
8,"Commerce Court,Victoria Hotel",0.000000,0.000000,0.040000,0.00,0.00,0.02,0.00,0.00,0.00,...,0.020000,0.000000,0.00,0.000000,0.000000,0.02,0.02,0.00,0.00,0.00
9,"Design Exchange,Toronto Dominion Centre",0.000000,0.000000,0.040000,0.00,0.00,0.02,0.00,0.00,0.02,...,0.020000,0.000000,0.00,0.000000,0.000000,0.02,0.02,0.00,0.00,0.00


Let's confirm the new size

In [61]:
Downtown_Toronto_grouped.shape

(18, 181)

Let's print each borough along with the top 5 most common venues.

In [68]:
num_top_venues = 5

for hood in Downtown_Toronto_grouped['Borough']:
    print("----"+hood+"----")
    temp = Downtown_Toronto_grouped[Downtown_Toronto_grouped['Borough']== hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0                 Café  0.08
1  American Restaurant  0.06
2          Coffee Shop  0.06
3     Asian Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park----
          venue  freq
0          Café  0.06
1   Coffee Shop  0.06
2      Beer Bar  0.06
3        Bakery  0.04
4  Cocktail Bar  0.04


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
             venue  freq
0      Coffee Shop  0.13
1  Harbor / Marina  0.13
2             Café  0.13
3     Dance Studio  0.07
4           Garden  0.07


----Cabbagetown,St. James Town----
                 venue  freq
0                 Park  0.08
1                 Café  0.05
2                Diner  0.05
3  Japanese Restaurant  0.05
4            Gastropub  0.05


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.12
1                 Café  0.06
2  Japanese Restaurant  0.04
3   Italian Restaurant  

Let's put that into a pandas dataframe.First, let's write a function to sort the venues in descending order.

In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [72]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = Downtown_Toronto_grouped['Borough']

for ind in np.arange(Downtown_Toronto_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_Toronto_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head(10)

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,American Restaurant,Coffee Shop,Hotel,Concert Hall,Asian Restaurant,Steakhouse,Gastropub,Burger Joint,Bookstore
1,Berczy Park,Café,Beer Bar,Coffee Shop,Hotel,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Pizza Place,Concert Hall
2,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Harbor / Marina,Café,Airport,Garden,Scenic Lookout,Sculpture Garden,Park,Dance Studio,Track
3,"Cabbagetown,St. James Town",Park,Coffee Shop,Gastropub,Café,Diner,Japanese Restaurant,Deli / Bodega,Pet Store,Performing Arts Venue,Pool
4,Central Bay Street,Coffee Shop,Café,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Tea Room,Yoga Studio,Sandwich Place,Ramen Restaurant,Portuguese Restaurant
5,"Chinatown,Grange Park,Kensington Market",Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Caribbean Restaurant,Coffee Shop,Dessert Shop,Bakery,Comfort Food Restaurant,Vietnamese Restaurant,Fish Market
6,Christie,Café,Korean Restaurant,Cocktail Bar,Grocery Store,Coffee Shop,Indian Restaurant,Pizza Place,Comedy Club,Cupcake Shop,Restaurant
7,Church and Wellesley,Coffee Shop,Gastropub,Gay Bar,Men's Store,Restaurant,Dance Studio,Thai Restaurant,Café,Park,Pizza Place
8,"Commerce Court,Victoria Hotel",Café,Coffee Shop,Hotel,Steakhouse,Restaurant,Gym,Gastropub,American Restaurant,Bakery,Deli / Bodega
9,"Design Exchange,Toronto Dominion Centre",Café,Coffee Shop,Hotel,Restaurant,American Restaurant,Concert Hall,Steakhouse,Gastropub,Deli / Bodega,Pub


Now Clustering boroughs by running k-means to cluster the neighborhood into 5 clusters.

In [90]:
# set number of clusters
kclusters = 5

Downtown_Toronto_grouped_clustering = Downtown_Toronto_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
boroughs_venues_sorted.head()

,Cluster Labels,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,"Adelaide,King,Richmond",Café,American Restaurant,Coffee Shop,Hotel,Concert Hall,Asian Restaurant,Steakhouse,Gastropub,Burger Joint,Bookstore
1,0,Berczy Park,Café,Beer Bar,Coffee Shop,Hotel,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Pizza Place,Concert Hall
2,1,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Harbor / Marina,Café,Airport,Garden,Scenic Lookout,Sculpture Garden,Park,Dance Studio,Track
3,0,"Cabbagetown,St. James Town",Park,Coffee Shop,Gastropub,Café,Diner,Japanese Restaurant,Deli / Bodega,Pet Store,Performing Arts Venue,Pool
4,4,Central Bay Street,Coffee Shop,Café,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Tea Room,Yoga Studio,Sandwich Place,Ramen Restaurant,Portuguese Restaurant


Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [96]:
boroughs_venues_sorted.shape

(18, 12)

Creating a new version of dataframe by merging Downtown_Toronto and boroughs_venues_sorted in a final dataframe called Toronto_Neighbourhood. First by sorting Downtown_Toronto dataframe alphebetically by neighbourhood coluumns.

In [104]:
Downtown_Toronto=Downtown_Toronto.sort_values('Neighbourhood')
Downtown_Toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
14,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
13,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049
17,M6G,Downtown Toronto,Christie,43.669542,-79.422564
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
11,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817
10,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576


In [107]:
boroughs_venues_sorted.rename(columns={'Borough':'Neigbourhood'},inplace=True)
boroughs_venues_sorted.rename(columns={'Neigbourhood':'Neighbourhood'},inplace=True)

In [108]:
Toronto_Neighbourhood=Downtown_Toronto
Toronto_Neighbourhood =Toronto_Neighbourhood.join(boroughs_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
Toronto_Neighbourhood.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,3,Café,American Restaurant,Coffee Shop,Hotel,Concert Hall,Asian Restaurant,Steakhouse,Gastropub,Burger Joint,Bookstore
6,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Café,Beer Bar,Coffee Shop,Hotel,Cocktail Bar,Farmers Market,Bakery,Seafood Restaurant,Pizza Place,Concert Hall
14,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,1,Coffee Shop,Harbor / Marina,Café,Airport,Garden,Scenic Lookout,Sculpture Garden,Park,Dance Studio,Track
1,M4X,Downtown Toronto,"Cabbagetown,St. James Town",43.667967,-79.367675,0,Park,Coffee Shop,Gastropub,Café,Diner,Japanese Restaurant,Deli / Bodega,Pet Store,Performing Arts Venue,Pool
7,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,4,Coffee Shop,Café,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Tea Room,Yoga Studio,Sandwich Place,Ramen Restaurant,Portuguese Restaurant
13,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049,0,Café,Mexican Restaurant,Vegetarian / Vegan Restaurant,Caribbean Restaurant,Coffee Shop,Dessert Shop,Bakery,Comfort Food Restaurant,Vietnamese Restaurant,Fish Market
17,M6G,Downtown Toronto,Christie,43.669542,-79.422564,4,Café,Korean Restaurant,Cocktail Bar,Grocery Store,Coffee Shop,Indian Restaurant,Pizza Place,Comedy Club,Cupcake Shop,Restaurant
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,4,Coffee Shop,Gastropub,Gay Bar,Men's Store,Restaurant,Dance Studio,Thai Restaurant,Café,Park,Pizza Place
11,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817,3,Café,Coffee Shop,Hotel,Steakhouse,Restaurant,Gym,Gastropub,American Restaurant,Bakery,Deli / Bodega
10,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,3,Café,Coffee Shop,Hotel,Restaurant,American Restaurant,Concert Hall,Steakhouse,Gastropub,Deli / Bodega,Pub


Now visualizing the result by Folium. In order to do that we need to get the coordinate of downtown Toronto by using geocoder and Nomination function from that module.

In [111]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geographiclib-1.49   | 32 KB     | ##################################### | 100% 
geopy-1.20.0         | 57 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [113]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [116]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_Neighbourhood['Latitude'], Toronto_Neighbourhood['Longitude'], Toronto_Neighbourhood['Neighbourhood'], Toronto_Neighbourhood['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters